In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [74]:
# train 데이터 불러오기
train = pd.read_csv('dataset/train_data.csv')

# test 데이터 불러오기
test = pd.read_csv('dataset/test_data.csv')

In [55]:
train

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0
...,...,...,...,...,...,...,...,...
2458,2.28,50.20,0.7,29.53,2416.0,66.84,3.34,7
2459,2.04,46.94,0.7,26.34,2155.0,63.94,3.20,7
2460,1.19,35.74,0.7,15.39,1259.0,53.99,2.70,7
2461,1.21,36.00,0.7,15.64,1280.0,54.22,2.71,7


In [56]:
test

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0
...,...,...,...,...,...,...,...,...
7384,2.12,48.08,0.7,27.45,2246.0,64.96,3.25,7
7385,1.48,39.63,0.7,19.19,1570.0,57.44,2.87,7
7386,1.56,40.61,0.7,20.15,1649.0,58.32,2.92,7
7387,1.59,40.99,0.7,20.52,1679.0,58.66,2.93,7


In [68]:
train = pd.concat([train,test], ignore_index = True)
train

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,1.59,41.00,0.7,20.53,1680.0,58.67,2.93,0
1,2.97,59.28,0.7,38.40,3142.0,74.91,3.75,0
2,1.91,45.29,0.7,24.73,2023.0,62.48,3.12,0
3,2.37,51.33,0.7,30.63,2506.0,67.84,3.39,0
4,1.90,45.21,0.7,24.65,2017.0,62.41,3.12,0
...,...,...,...,...,...,...,...,...
32014,2.12,48.08,0.7,27.45,2246.0,64.96,3.25,7
32015,1.48,39.63,0.7,19.19,1570.0,57.44,2.87,7
32016,1.56,40.61,0.7,20.15,1649.0,58.32,2.92,7
32017,1.59,40.99,0.7,20.52,1679.0,58.66,2.93,7


In [112]:
from keras.layers.rnn import TimeDistributed
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.backend import clear_session
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

clear_session()

train_1 = train[train['type'] == 1].iloc[:, :-1]
test_1 = test[test['type'] == 1].iloc[:, :-1]

# 정규화
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_1)
test_scaled = scaler.transform(test_1)

# reshape
train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, train_scaled.shape[1])
test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

# 이 부분만 변경함
def ae_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = Dense(64, activation='relu')(inputs)
    L2 = Dense(32, activation='relu')(L1)
    L3 = Dense(16, activation='relu')(L2)
    encoded = Dense(8, activation='relu')(L3)
    L5 = Dense(16, activation='relu')(encoded)
    L6 = Dense(32, activation='relu')(L5)
    L7 = Dense(64, activation='relu')(L6)
    decoded = Dense(X.shape[2], activation=None)(L7)

    model = Model(inputs, decoded)

    return model


# # LSTM AE 모델 구축
# def ae_model(X):
#     inputs = Input(shape=(X.shape[1], X.shape[2]))
#     L1 = LSTM(64, activation='relu', return_sequences=True,
#             kernel_regularizer=regularizers.l2(0.00))(inputs)
#     L2 = LSTM(10, activation='relu', return_sequences=False)(L1)
#     L3 = RepeatVector(X.shape[1])(L2)
#     L4 = LSTM(10, activation='relu', return_sequences=True)(L3)
#     L5 = LSTM(64, activation='relu', return_sequences=True,
#             kernel_regularizer=regularizers.l2(0.00))(L4)
#     output = TimeDistributed(Dense(X.shape[2]))(L5)

#     model = Model(inputs, output)
#     return model


# 모델 컴파일
model = ae_model(train_scaled)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

# 모델 학습
history = model.fit(train_scaled, train_scaled, epochs=20, batch_size=32, validation_split=0.1).history

# 시각화
#fig, ax = plt.subplots(figsize=(14, 6), dpi=400)
#ax.plot(history['loss'], 'b', label='Train')
#ax.plot(history['val_loss'], 'r', label='Validation')
#ax.set_title('Model loss')
#ax.set_ylabel('Loss (mse)')
#ax.set_xlabel('Epochs')
#ax.legend()
#plt.show()

# train 재구성 오차를 계산합니다.
train_pred = model.predict(train_scaled)
train_pred = train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
train_pred = pd.DataFrame(train_pred, columns=train_1.columns)
train_pred.index = train_1.index

scored = pd.DataFrame(index=train_1.index)
Xtrain = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[2])
scored['Loss_mse'] = np.mean(np.abs(train_pred - Xtrain), axis=1)

# test 재구성 오차
threshold = np.percentile(scored, 100)     

test_pred = model.predict(test_scaled)
test_pred = test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
test_pred = pd.DataFrame(test_pred, columns=test_1.columns)
test_pred.index = test_1.index

test_scored = pd.DataFrame(index=test_1.index)
Xtest = test_scaled.reshape(test_scaled.shape[0], test_scaled.shape[2])
test_scored['Loss_mse'] = np.mean(np.abs(test_pred - Xtest), axis=1)
test_scored['Threshold'] = threshold
test_scored['Anomaly'] = test_scored['Loss_mse'] > test_scored['Threshold']

test_scored['Anomaly'] = test_scored['Anomaly'].replace(False, 0)
test_scored['Anomaly'] = test_scored['Anomaly'].replace(True, 1)
test_scored.head()

Epoch 1/20
11/11 [==============================] - 1s 41ms/step - loss: 0.2013 - val_loss: 0.1227
Epoch 2/20
11/11 [==============================] - 0s 7ms/step - loss: 0.1160 - val_loss: 0.0351
Epoch 3/20
11/11 [==============================] - 0s 6ms/step - loss: 0.0314 - val_loss: 0.0193
Epoch 4/20
11/11 [==============================] - 0s 7ms/step - loss: 0.0158 - val_loss: 0.0088
Epoch 5/20
11/11 [==============================] - 0s 7ms/step - loss: 0.0120 - val_loss: 0.0054
Epoch 6/20
11/11 [==============================] - 0s 8ms/step - loss: 0.0085 - val_loss: 0.0028
Epoch 7/20
11/11 [==============================] - 0s 7ms/step - loss: 0.0062 - val_loss: 7.2670e-04
Epoch 8/20
11/11 [==============================] - 0s 6ms/step - loss: 0.0052 - val_loss: 4.7489e-04
Epoch 9/20
11/11 [==============================] - 0s 6ms/step - loss: 0.0047 - val_loss: 4.5020e-04
Epoch 10/20
11/11 [==============================] - 0s 7ms/step - loss: 0.0040 - val_loss: 2.1113e-04
Ep

,Loss_mse,Threshold,Anomaly
1296,0.001975,0.028366,0
1297,0.002311,0.028366,0
1298,0.001349,0.028366,0
1299,0.002228,0.028366,0
1300,0.001463,0.028366,0


In [113]:
test_scored['Anomaly'].value_counts()

0    1107
Name: Anomaly, dtype: int64

In [114]:
# 결과
result = pd.read_csv('dataset/answer_sample.csv')
result_1 = result[result['type'] == 1]
result_1['label'] = test_scored['Anomaly'].values

In [115]:
clear_session()

train_2 = train[train['type'] == 2].iloc[:, :-1]
test_2 = test[test['type'] == 2].iloc[:, :-1]

# 정규화
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_2)
test_scaled = scaler.transform(test_2)

# reshape
train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, train_scaled.shape[1])
test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

# 모델 컴파일
model = ae_model(train_scaled)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

# 모델 학습
history = model.fit(train_scaled, train_scaled, epochs=40, batch_size=32, validation_split=0.1).history

# train 재구성 오차를 계산합니다.
train_pred = model.predict(train_scaled)
train_pred = train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
train_pred = pd.DataFrame(train_pred, columns=train_2.columns)
train_pred.index = train_2.index

scored = pd.DataFrame(index=train_2.index)
Xtrain = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[2])
scored['Loss_mse'] = np.mean(np.abs(train_pred - Xtrain), axis=1)


# test 재구성 오차
threshold = np.percentile(scored, 100)     

test_pred = model.predict(test_scaled)
test_pred = test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
test_pred = pd.DataFrame(test_pred, columns=test_2.columns)
test_pred.index = test_2.index

test_scored = pd.DataFrame(index=test_2.index)
Xtest = test_scaled.reshape(test_scaled.shape[0], test_scaled.shape[2])
test_scored['Loss_mse'] = np.mean(np.abs(test_pred - Xtest), axis=1)
test_scored['Threshold'] = threshold
test_scored['Anomaly'] = test_scored['Loss_mse'] > test_scored['Threshold']

test_scored['Anomaly'] = test_scored['Anomaly'].replace(False, 0)
test_scored['Anomaly'] = test_scored['Anomaly'].replace(True, 1)
test_scored.head()

Epoch 1/40
11/11 [==============================] - 2s 39ms/step - loss: 0.2451 - val_loss: 0.1931
Epoch 2/40
11/11 [==============================] - 0s 7ms/step - loss: 0.1405 - val_loss: 0.0568
Epoch 3/40
11/11 [==============================] - 0s 6ms/step - loss: 0.0372 - val_loss: 0.0182
Epoch 4/40
11/11 [==============================] - 0s 7ms/step - loss: 0.0097 - val_loss: 0.0066
Epoch 5/40
11/11 [==============================] - 0s 7ms/step - loss: 0.0053 - val_loss: 0.0026
Epoch 6/40
11/11 [==============================] - 0s 7ms/step - loss: 0.0017 - val_loss: 5.4933e-04
Epoch 7/40
11/11 [==============================] - 0s 7ms/step - loss: 5.1547e-04 - val_loss: 2.7430e-04
Epoch 8/40
11/11 [==============================] - 0s 7ms/step - loss: 2.8451e-04 - val_loss: 1.7053e-04
Epoch 9/40
11/11 [==============================] - 0s 9ms/step - loss: 1.6941e-04 - val_loss: 6.4419e-05
Epoch 10/40
11/11 [==============================] - 0s 7ms/step - loss: 9.8380e-05 - val

,Loss_mse,Threshold,Anomaly
2403,0.001271,0.004077,0
2404,0.001215,0.004077,0
2405,0.001479,0.004077,0
2406,0.000284,0.004077,0
2407,0.001260,0.004077,0


In [116]:
test_scored['Anomaly'].value_counts()

0    1071
1      27
Name: Anomaly, dtype: int64

In [117]:
# 결과
result_2 = result[result['type'] == 2]
result_2['label'] = test_scored['Anomaly'].values

In [118]:
clear_session()

train_3 = train[train['type'] == 3].iloc[:, :-1]
test_3 = test[test['type'] == 3].iloc[:, :-1]

# 정규화
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_3)
test_scaled = scaler.transform(test_3)

# reshape
train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, train_scaled.shape[1])
test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

# 모델 컴파일
model = ae_model(train_scaled)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

# 모델 학습
history = model.fit(train_scaled, train_scaled, epochs=40, batch_size=32, validation_split=0.1).history

# train 재구성 오차를 계산합니다.
train_pred = model.predict(train_scaled)
train_pred = train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
train_pred = pd.DataFrame(train_pred, columns=train_3.columns)
train_pred.index = train_3.index

scored = pd.DataFrame(index=train_3.index)
Xtrain = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[2])
scored['Loss_mse'] = np.mean(np.abs(train_pred - Xtrain), axis=1)

# test 재구성 오차
threshold = np.percentile(scored, 100)     

test_pred = model.predict(test_scaled)
test_pred = test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
test_pred = pd.DataFrame(test_pred, columns=test_3.columns)
test_pred.index = test_3.index

test_scored = pd.DataFrame(index=test_3.index)
Xtest = test_scaled.reshape(test_scaled.shape[0], test_scaled.shape[2])
test_scored['Loss_mse'] = np.mean(np.abs(test_pred - Xtest), axis=1)
test_scored['Threshold'] = threshold
test_scored['Anomaly'] = test_scored['Loss_mse'] > test_scored['Threshold']

test_scored['Anomaly'] = test_scored['Anomaly'].replace(False, 0)
test_scored['Anomaly'] = test_scored['Anomaly'].replace(True, 1)
test_scored.head()

Epoch 1/40
9/9 [==============================] - 1s 50ms/step - loss: 0.2753 - val_loss: 0.1706
Epoch 2/40
9/9 [==============================] - 0s 8ms/step - loss: 0.1745 - val_loss: 0.0728
Epoch 3/40
9/9 [==============================] - 0s 7ms/step - loss: 0.0552 - val_loss: 0.0247
Epoch 4/40
9/9 [==============================] - 0s 8ms/step - loss: 0.0228 - val_loss: 0.0126
Epoch 5/40
9/9 [==============================] - 0s 8ms/step - loss: 0.0099 - val_loss: 0.0093
Epoch 6/40
9/9 [==============================] - 0s 7ms/step - loss: 0.0080 - val_loss: 0.0068
Epoch 7/40
9/9 [==============================] - 0s 7ms/step - loss: 0.0045 - val_loss: 0.0034
Epoch 8/40
9/9 [==============================] - 0s 7ms/step - loss: 0.0027 - val_loss: 0.0021
Epoch 9/40
9/9 [==============================] - 0s 8ms/step - loss: 0.0014 - val_loss: 8.8833e-04
Epoch 10/40
9/9 [==============================] - 0s 8ms/step - loss: 5.8404e-04 - val_loss: 4.1864e-04
Epoch 11/40
9/9 [=========

,Loss_mse,Threshold,Anomaly
3501,0.001913,0.015217,0
3502,0.002101,0.015217,0
3503,0.001396,0.015217,0
3504,0.001601,0.015217,0
3505,0.001329,0.015217,0


In [119]:
test_scored['Anomaly'].value_counts()

0    891
1     27
Name: Anomaly, dtype: int64

In [120]:
# 결과
result_3 = result[result['type'] == 3]
result_3['label'] = test_scored['Anomaly'].values

In [121]:
clear_session()

train_4 = train[train['type'].isin([0, 4, 5, 6, 7])].iloc[:, :-1]
test_4 = test[test['type'].isin([0, 4, 5, 6, 7])].iloc[:, :-1]

# 정규화
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_4)
test_scaled = scaler.transform(test_4)

# reshape
train_scaled = train_scaled.reshape(train_scaled.shape[0], 1, train_scaled.shape[1])
test_scaled = test_scaled.reshape(test_scaled.shape[0], 1, test_scaled.shape[1])

# 모델 컴파일
model = ae_model(train_scaled)
model.compile(optimizer=Adam(lr=0.001), loss='mse')

# 모델 학습
history = model.fit(train_scaled, train_scaled, epochs=20, batch_size=32, validation_split=0.1).history

# train 재구성 오차를 계산합니다.
train_pred = model.predict(train_scaled)
train_pred = train_pred.reshape(train_pred.shape[0], train_pred.shape[2])
train_pred = pd.DataFrame(train_pred, columns=train_4.columns)
train_pred.index = train_4.index

scored = pd.DataFrame(index=train_4.index)
Xtrain = train_scaled.reshape(train_scaled.shape[0], train_scaled.shape[2])
scored['Loss_mse'] = np.mean(np.abs(train_pred - Xtrain), axis=1)

# test 재구성 오차
threshold = np.percentile(scored, 100)     

test_pred = model.predict(test_scaled)
test_pred = test_pred.reshape(test_pred.shape[0], test_pred.shape[2])
test_pred = pd.DataFrame(test_pred, columns=test_4.columns)
test_pred.index = test_4.index

test_scored = pd.DataFrame(index=test_4.index)
Xtest = test_scaled.reshape(test_scaled.shape[0], test_scaled.shape[2])
test_scored['Loss_mse'] = np.mean(np.abs(test_pred - Xtest), axis=1)
test_scored['Threshold'] = threshold
test_scored['Anomaly'] = test_scored['Loss_mse'] > test_scored['Threshold']

test_scored['Anomaly'] = test_scored['Anomaly'].replace(False, 0)
test_scored['Anomaly'] = test_scored['Anomaly'].replace(True, 1)
test_scored.head()

Epoch 1/20
40/40 [==============================] - 2s 12ms/step - loss: 0.1289 - val_loss: 0.0116
Epoch 2/20
40/40 [==============================] - 0s 4ms/step - loss: 0.0034 - val_loss: 0.0011
Epoch 3/20
40/40 [==============================] - 0s 4ms/step - loss: 3.8326e-04 - val_loss: 1.7765e-04
Epoch 4/20
40/40 [==============================] - 0s 4ms/step - loss: 7.0571e-05 - val_loss: 4.4416e-05
Epoch 5/20
40/40 [==============================] - 0s 4ms/step - loss: 2.4399e-05 - val_loss: 2.3166e-05
Epoch 6/20
40/40 [==============================] - 0s 4ms/step - loss: 1.3208e-05 - val_loss: 1.0918e-05
Epoch 7/20
40/40 [==============================] - 0s 4ms/step - loss: 6.6000e-06 - val_loss: 5.8082e-06
Epoch 8/20
40/40 [==============================] - 0s 3ms/step - loss: 3.8787e-06 - val_loss: 3.5894e-06
Epoch 9/20
40/40 [==============================] - 0s 4ms/step - loss: 2.7781e-06 - val_loss: 2.3961e-06
Epoch 10/20
40/40 [==============================] - 0s 5ms/s

,Loss_mse,Threshold,Anomaly
0,0.000541,0.00399,0
1,0.000759,0.00399,0
2,0.000434,0.00399,0
3,0.000889,0.00399,0
4,0.000405,0.00399,0


In [122]:
test_scored['Anomaly'].value_counts()

0    3973
1     293
Name: Anomaly, dtype: int64

## 결과 저장

In [123]:
# 결과
result_0 = result[result['type'].isin([0, 4, 5, 6, 7])]
result_0['label'] = test_scored['Anomaly'].values

In [124]:
final = pd.concat([result_0, result_1], axis=0).sort_index()
final = pd.concat([final, result_2], axis=0).sort_index()
final = pd.concat([final, result_3], axis=0).sort_index()

In [125]:
final.to_csv('autoencoder_lstm.csv', index=False)

## 주은님꺼랑 비교

In [128]:
# 비교
joo = pd.read_csv('autoencoder_dense.csv')

In [129]:
cnt = 0
diff_idx = []
for i in range(len(list(joo['label'].values))):
    if list(joo['label'].values)[i] != list(final['label'].values)[i]:
        cnt += 1
        diff_idx.append(i)
cnt

5

In [130]:
final.iloc[diff_idx, :]

,type,label
2755,2,0
2782,2,0
2887,2,0
3905,3,0
4378,3,1


In [21]:
final[final['label'] == 1]['label'].index

Int64Index([ 585,  586,  587,  588,  589,  590,  591,  592,  593,  594,
            ...
            6796, 6797, 6798, 6799, 6800, 7089, 7342, 7348, 7359, 7360],
           dtype='int64', length=342)

In [22]:
joo[joo['label'] == 1]['label'].index

Int64Index([ 585,  586,  587,  588,  589,  590,  591,  592,  593,  594,
            ...
            6796, 6797, 6798, 6799, 6800, 7089, 7342, 7348, 7359, 7360],
           dtype='int64', length=350)